In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import fitparse as fp
import csv
import pytz
import datetime
from pathlib import Path
import os
import shutil
from pathlib import Path
import re

In [2]:
def print_record(data):
    """
    test method to print out the data in a record
    """
    for record in data:
        for field in record:
            if field.units:
                print (" * %s: %s %s" % (
                    field.name, field.value, field.units))
            else:
                print (" * %s: %s" % (field.name, field.value))

In [3]:
def get_time(record):
    """
    extract the time from data. Note for heart rate data the timestamp is int(16) up to 65536 ~ 18hours
    This will overflow and needs normalised so it wont give a crazy result on overflow.

    Every now and then there will be a timestamp giving the right date and time. 
    Heart rate data is timestamped on a rolling seconds counter. Need to set the first timestamp_16 to the
    global time then work out the next time based on the difference between timestamp_16s adding that to global time
    """
    new_timestamp_16 = None
    
    # difference between successive timestamp_16
    delta_diff = 0
    global timestamp_16
    global current_timestamp

    for r in record:
        if r.field is not None:
            if r.name == 'timestamp_16':
                new_timestamp_16 = r.value

    # if the timestamp is none then set it to the global time
    if new_timestamp_16 is not None:
        if timestamp_16 is None:
            timestamp_16 = new_timestamp_16
        
        delta_diff = new_timestamp_16 - timestamp_16
        
        # deal with overflow
        if delta_diff < 0:
            delta_diff += 65536
        
        current_timestamp += datetime.timedelta(seconds=delta_diff)
        timestamp_16 = new_timestamp_16
        #print(current_timestamp)
        return current_timestamp

In [4]:
def get_heartrate(record):
    """
    Return the heart rate from the record
    """
    for r in record:
        if r.name == 'heart_rate':
            return r.value

In [5]:
def output_messages(fitfile):
    """
    Go through the fitfile and extract time and heart rate data
    """
    messages = fitfile.get_messages()

    current_timestamp = datetime.datetime.now()
    timestamp_16 = None

    hrdata = []
    timestamp = []

    for record in messages:
    # Go through all the data entries in this record

        # Extract the global time stamp
        if record.name == 'monitoring_info':
            for r in record.fields:
                if r.field is not None:
                    if r.field.name == 'local_timestamp':
                        current_timestamp = r.value
                        timestamp_16 = None
                        #print("Timestamp: {}".format(current_timestamp))

        # get the heart rate data
        if record.name == 'monitoring':
            for r in record:
                if r.field is not None:
                    if r.field.name =='heart_rate':
                        time = get_time(record)
                        hr = get_heartrate(record)

                        timestamp.append(time)
                        hrdata.append(hr)
                        #print("Time: {}   Heart: {}".format(time,hr))

In [6]:
def output_csv(time, hr, fi):
    """
    Write the extracted time and heart rate data to csv fiile
    """
    data = [time, hr]

    csvFile = open(fi,'w')
    with csvFile:
        print("Writing csv to: {}".format(fi))
        writer = csv.writer(csvFile, lineterminator='\n') # need lineterminator to prevent extra linebreaks
        writer.writerow(["Date time", "Heart rate BPM"])
        for i in range(len(time)):
            writer.writerow([time[i],hr[i]])

In [7]:

def copy_files_without_email(source, destination):
    r = []
    for root, dirs, files in os.walk(source):
        for name in files:
            filepath = root + os.sep + name
            if filepath.endswith(".fit"):
                name = Path(filepath)
                #print(name.stem)
                no_email = re.findall(r'(?<=_).*$', name.stem) # look for anything after _
                #print(no_email[0])
                if len(no_email)  is not None:
                    new_name = no_email[0] + name.suffix
                    dest = destination + new_name
                    #print(dest)
                    shutil.copy(filepath, destination+new_name)
                

In [8]:
raw_data_dir = Path(r"Garmin_data/data")
dest_data_dir = Path(r"Garmin_data/fit/")

In [9]:
#copy_files_without_email(raw_data_dir, dest_data_dir)

In [10]:
#os.listdir(dest_data_dir)

In [11]:
#grab last 4 characters of the file name:
def filenumber_beginning(x):
    # get file name (xxx.fit) and split on .fit
    # if filename has xxx_yyy.fit then split on _yyy.fit
    # return xxx
    filename = os.path.splitext(x)[0]
    
    if '_' in filename:
        filename = filename.split('_')[0]

    file_out = int(filename)

    return(file_out)

file_list = os.listdir(dest_data_dir)

s = sorted(file_list, key = filenumber_beginning) 

print(f"files {len(s)}")

files 8523


In [82]:
parse_file = dest_data_dir.joinpath(s[100])
fitfile = fp.FitFile(str(parse_file)) # needs string file path and not windows path that sorted produces

In [66]:
xt310 = "fr310xt_4t"
xt310_no = "1080"
xt735 = "2158"
watch_dev_no = [xt310, xt310_no, xt735]

In [67]:
message_dict = {
"file_id" : "file_id",
"record" : "record",
"device_info" : "device_info"
}

In [68]:
di = fitfile.get_messages(message_dict["file_id"])
product = []
for r in di:
    print(f"-{r}")
    product = list(filter( lambda obj: obj.name=="garmin_product", r))

print(product[0].value)

-file_id (#0)
2158


for 310_xt it is all records. 
Pull out `timestamp: 2017-01-18 19:31:28` and `heart_rate: 160 [bpm]`

In [70]:
recs = fitfile.get_messages(message_dict["record"])
heartrate = []
timestamp = []

for record in recs:
    hr = None
    time = None
    hr = list(filter(lambda a: a.name == 'heart_rate', record))
    time =  list(filter(lambda a: a.name == 'timestamp', record))

    if hr is not None and time is not None:    
        heartrate.append(hr)
        timestamp.append(time)

for h in heartrate:
    print(h[0].value)

for t in timestamp:
    print(t[0].value)
    print(type(t[0].value))

In [91]:
for file in s:
    print(dest_data_dir.joinpath(file))
    try:
        ff = fp.FitFile(str(dest_data_dir.joinpath(file)))
        id = get_device(ff)
        if id is xt310_no or id is xt310:
            print(id)
        else:
            print(f"-{id}")
    except Exception:
        print(Exception)

Garmin_data\fit\4946143282.fit
fr310xt_4t
Garmin_data\fit\5003492067.fit
fr310xt_4t
Garmin_data\fit\5003492440.fit
fr310xt_4t
Garmin_data\fit\5079205929.fit
fr310xt_4t
Garmin_data\fit\5127558264.fit
fr310xt_4t
Garmin_data\fit\5216580561.fit
fr310xt_4t
Garmin_data\fit\5261209558.fit
fr310xt_4t
Garmin_data\fit\5293790961.fit
fr310xt_4t
Garmin_data\fit\5293791276.fit
fr310xt_4t
Garmin_data\fit\5496654950.fit
fr310xt_4t
Garmin_data\fit\5496655249.fit
fr310xt_4t
Garmin_data\fit\5496655506.fit
fr310xt_4t
Garmin_data\fit\5593033536.fit
fr310xt_4t
Garmin_data\fit\5715608206.fit
fr310xt_4t
Garmin_data\fit\5715608547.fit
fr310xt_4t
Garmin_data\fit\5715608876.fit
fr310xt_4t
Garmin_data\fit\5715609115.fit
fr310xt_4t
Garmin_data\fit\5715609316.fit
fr310xt_4t
Garmin_data\fit\5745099991.fit
fr310xt_4t
Garmin_data\fit\5745100238.fit
fr310xt_4t
Garmin_data\fit\5919004141.fit
fr310xt_4t
Garmin_data\fit\5919004420.fit
fr310xt_4t
Garmin_data\fit\5919004683.fit
fr310xt_4t
Garmin_data\fit\5919004970.fit
fr3

KeyboardInterrupt: 

In [56]:
di = fitfile.get_messages(message_dict["file_id"])
print(str(di))
for r in di:
    print(f"-{r}")
    for x in r:
        print(f"    {x}")

<generator object FitFile.get_messages at 0x0000023662E85F40>
-file_id (#0)
    garmin_product: fr310xt_4t
    manufacturer: garmin
    number: None
    serial_number: 3871167223
    time_created: 2017-01-18 18:49:35
    type: activity


In [ ]:
di = fitfile.get_messages()
for r in di:
    print(f"-{r}")
    for x in r:
        print(f"    {x}")

In [ ]:
#choose the right directory here fit files are located
dir = "data/Fit_data/"
testdir = "C:/temp/data/test"
datadir = dir

UTC = pytz.UTC
GMT = pytz.timezone('Europe/London')

# heart rate data list
hrdata = []
# corresponding timestamp
timestamp = []

# globals to get the time data
current_timestamp = datetime.datetime.now()
timestamp_16 = None

In [ ]:
# get the fit files in order (maybe)
files = sorted(Path(datadir).glob('*.fit'))

for f in files:

    ff = datadir+'/'+f.

    fitfile = fitparse.FitFile(ff)

    print("converting {}".format(f.name))

    output_messages(fitfile)

fi = datadir+'/'+'op.csv'
output_csv(timestamp, hrdata, fi)


In [ ]:
np.uint16(np.iinfo(np.uint16).max) # unit16 max

65535